## 第五章 误差反向传播法

1、正向传播与反向传播的特点与区别：正向传播将结果从左往右传播，反向传播将局部的导数从沿正方向的反向传播。
2、反向传播遵循链式的求导法则，并且分为加法节点和乘法节点，加法节点导数为1，反向传播只乘1；乘法节点乘以节点另一侧对应的导数。

In [7]:
class  Mullayer:#将乘法层实现为一类。
    def __init__(self):#用于记录保存正向传播的输入值。
        self.x=None
        self.y=None
        
    def forward(self,x,y):
        self.x=x
        self.y=y
        out=x*y
        
        return out
    
    def backward(self,dout):
        dx=dout*self.y
        dy=dout*self.x
        
        return dx,dy
    
class Addlayer:#加法层的实现
    def __init__(self):
        pass
    
    def forward(self,x,y):
        out=x+y
        return out
    def backward(self,dout):
        dx=dout*1
        dy=dout*1
        return dx,dy
    
    
    


In [9]:
apple=100;#forward
apple_num=2;
tax=1.1;
apple_mullayer=Mullayer()
tax_mullayer=Mullayer()

apple_price=apple_mullayer.forward(apple,apple_num)
price=tax_mullayer.forward(apple_price,tax)


dprice=1#backward
dapple,dtax=tax_mullayer.backward(dprice)
dapple,dnum=apple_mullayer.backward(dapple)
print(dapple,dtax,dapple,dnum)

2.2 200 2.2 110.00000000000001


## 激活函数层的实现

In [2]:
class Relu:
    def _init_(self):
        self.mask=None
        
    def forward(self,x):
        self.mask=(x<=0)
        out=x.copy()#将x的列表复制了一次，内容同x的一致
        out[self.mask]=0#将为Ture的标记为0
        
        return out
    def backward(self,dout):
        dout[self.mask]=0
        dx=dout
        
        return dx

In [8]:
class sigmoid:
    def _init_(self):
        self.out=None
    def forward(self,x):
        out=1/(1+np.exp(-x))
        self.out=out
        
        return out
    def backward(self,dout):
        dx=dout*(1-self.out)*self.out
        
        return dx
    
        


In [10]:
class Affine:
    def _init_(self,w,b):
        self.w=w
        self.b=b
        self.x=None
        self.dw=None
        self.db=None
    def forward(self,x):
        self.x=x
        out=np.dot(x,self.w)+self.b
        
        return out
    def backward(self,dout):
        dx=np.dot(dout,self.w.T)
        self.dw=np.dot(self.x.T,dout)
        self.db=np.sum(dout,axis=0)#按行项求和
        
        return dx

## Softmax-with-Loss层，利用softmax输出之后，计算交叉熵误差，可以得到反向传播满意的结果，即输出与监督标签的误差。从而进行反馈。

In [11]:
class SoftmaxwithLoss:
    def _init_(self):
        self.loss=None
        self.y=None
        self.t=None
    def forward(self,x,t):
        self.t=t
        self.y=softmax(x)
        self.loss=cross_entropy_loss(self.y,self.t)
        
        return self.loss
    def backward(self,dout=1):
        batch_size=self.t.shape[0]
        dx=(self.y-self.t)/batch_size#传给前边是单个数据的误差，因此除以批量大小。
        return dx